In [320]:
import pandas as pd
import numpy as np
pd.options.display.float_format = "{:,.4f}".format

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
import math

import warnings
warnings.filterwarnings("ignore")

# Library of Tobias
from TA_utils import *

## Some Helper Functions

In [321]:
# Easy to read summary stats - monthly return data!

# Always plug in a DataFrame - never pd.Series
# From Mani
def summary_statistics_annualized(returns, annual_factor = 12):
    """This functions returns the summary statistics for the input total/excess returns passed
    into the function"""
    
    summary_statistics = pd.DataFrame(index=returns.columns)
    summary_statistics['Mean'] = returns.mean() * annual_factor
    summary_statistics['Vol'] = returns.std() * np.sqrt(annual_factor)
    summary_statistics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annual_factor)
    summary_statistics['Min'] = returns.min()
    summary_statistics['Max'] = returns.max()
    summary_statistics['Skewness'] = returns.skew()
    summary_statistics['Excess Kurtosis'] = returns.kurtosis()
    summary_statistics['VaR (0.05)'] = returns.quantile(.05, axis = 0)
    summary_statistics['CVaR (0.05)'] = returns[returns <= returns.quantile(.05, axis = 0)].mean()

    wealth_index = 1000*(1+returns).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_statistics['Max Drawdown'] = drawdowns.min()
    summary_statistics['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_statistics['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_statistics['Recovery'] = recovery_date
    
    return summary_statistics

In [322]:
# Draw the correlation matrix and visualize it
def corr_matrix(df):
    corrM = df.corr()
    return corrM
    
def visualize_corr_matrix(df):  
    corrM = corr_matrix(df)
    dims = (8, 6)
    fig, ax = pyplot.subplots(figsize=dims)
    sn.heatmap(data = corrM, ax=ax, cmap ='Reds', linewidth=.2, annot=True)

In [323]:
# From Mani
def tangency_weights(returns, cov_mat = 1, annual_factor = 12):
    """
    cov_mat = 0.5 if you want to regularize your covariance matrix by shrinking
    """
    if cov_mat ==1:
        cov_inv = np.linalg.inv((returns.cov() * annual_factor))
    else:
        cov = returns.cov()
        covmat_diag = np.diag(np.diag((cov)))
        covmat = cov_mat * cov + (1-cov_mat) * covmat_diag
        cov_inv = np.linalg.inv((covmat * annual_factor))  
        
    ones = np.ones(returns.columns.shape) 
    mu = returns.mean() * annual_factor
    scaling = 1/(np.transpose(ones) @ cov_inv @ mu)
    tangent_return = scaling*(cov_inv @ mu) 
    tangency_wts = pd.DataFrame(index = returns.columns, data = tangent_return, columns = ['Tangent Weights'] )
        
    return tangency_wts

In [324]:
def regression_based_performance(factor, ret_data, rf=0, period = 12, constant = True): 
    """
    if, reg = regression_based_performance(factor, ret_data, rf=0, period = 12, constant = True)
    then, beta = reg[0], alpha = reg[3], rsquared = reg[4]
    """
    if constant: # This ensures that there is an intercept in the model
        X = sm.tools.add_constant(factor)
    else:
        X = factor
    Y = ret_data

    model = sm.OLS(Y,X, missing='drop').fit()
    
    beta_df = pd.DataFrame(index= [ret_data.name])
    betaCols = [i+'Beta' for i in factor]
    
    if constant:
        alpha = model.params['const'] * period
        for i,x in enumerate(factor):
            beta_df[x] = model.params[i+1] #The first term is the intercept and the rest is the coefficients of the regressors
    else:
        alpha = 0
        for i,x in enumerate(factor):
            beta_df[x] = model.params[i]

    beta_df = beta_df.rename(columns = dict(zip(beta_df.columns,betaCols)))

    mean = ret_data.mean() * period
    tracking_error = (model.resid.std() * math.sqrt(period))

    if constant:
        treynor_ratio = (ret_data - rf).mean() * period/model.params[1]
        information_ratio = model.params[0] * period / tracking_error
    else:
        treynor_ratio = (ret_data - rf).mean() * period/model.params[0]
        
    rsquared = model.rsquared

    RegressionStats = pd.DataFrame({'Mean Return':mean, 'R-squared':rsquared, 'Alpha':alpha, 'Tracking Error':tracking_error,\
                          'Information Ratio':information_ratio, 'Treynor Ratio':treynor_ratio},index= [ret_data.name])
    
    return pd.concat([RegressionStats,beta_df], axis =1)

## Read Data
    MKT: Excess Return
    SMB: Small - Big (Size)
    HML: High - Low (Value)
    RMW: Robust - Weak (Profitabiliy)
    CMA: Conservative - Aggresive (Investment)
    UMD: Up - Down (Momentum) - 6th Factor

In [325]:
factor_exc_ret = pd.read_excel('factor_pricing_data.xlsx', sheet_name = 'factors (excess returns)', index_col = 0)
port_exc_ret = pd.read_excel('factor_pricing_data.xlsx', sheet_name = 'portfolios (excess returns)', index_col = 0)


## 2.1


In [326]:
summary_statistics_annualized(factor_exc_ret)[['Mean', 'Vol', 'Sharpe']]

,Mean,Vol,Sharpe
MKT,0.0846,0.1573,0.5376
SMB,0.0112,0.1005,0.1115
HML,0.0253,0.1102,0.2299
RMW,0.0465,0.0834,0.5578
CMA,0.0325,0.0734,0.4428
UMD,0.0609,0.1555,0.3918


## 2.2
    a) All factors have positive premium in the course of the entire sample. In other words they had positive expected excess return
    b) Since 2015, even though excess return is higher than the over the course of whole sample, SMB and HML did not have positive expected excess return (check below)

In [327]:
factor_exc_ret_since_2015 = factor_exc_ret.loc['2015':]
summary_statistics_annualized(factor_exc_ret_since_2015)[['Mean', 'Vol', 'Sharpe']]

,Mean,Vol,Sharpe
MKT,0.1117,0.1627,0.6865
SMB,-0.0084,0.0983,-0.0852
HML,-0.0258,0.1358,-0.1901
RMW,0.0486,0.0731,0.6645
CMA,0.0018,0.0870,0.0206
UMD,0.0065,0.1440,0.0451


## 2.3 - Correlation matrix
    a) Overall, factors are doing a very nice job in keeping the correlation small. It's even negative for the most of pairs. Other than the outlier correlation data between CMA and HML (corr = 0.6810), correlation is kept quite at a low level
    b) As pointed out in a) that CMA and HML have a high pairwise correlation. Removing one can be a good idea

In [328]:
corr_matrix(factor_exc_ret)

,MKT,SMB,HML,RMW,CMA,UMD
MKT,1.0000,0.2260,-0.2108,-0.2382,-0.3621,-0.1787
SMB,0.2260,1.0000,-0.0541,-0.4143,-0.0631,-0.0382
HML,-0.2108,-0.0541,1.0000,0.2298,0.6810,-0.2091
RMW,-0.2382,-0.4143,0.2298,1.0000,0.1350,0.0789
CMA,-0.3621,-0.0631,0.6810,0.1350,1.0000,0.0145
UMD,-0.1787,-0.0382,-0.2091,0.0789,0.0145,1.0000


## 2.4 
    a) With the highest weight 0.3693 CMA is the most weighted factor. It's followed by RMW with 0.3074 and MKT with 0.02039. The least important factors are HML with a weight of -0.0618 and SMB with a weigt of 0.0874
    b) It seems like the two factors with the lowest mean returns (SMB and HML) are not put much weights by the MV optimization
    c) When some factors are excluded in the model, standing of factors in terms of importance seems to change dramatically. For instance, HML was the least important when 6 factors were together in the model. After removing CMA and RMW, HML becomes the one of the most important factors. Another example is SMB was more important than HML and very similar to UMD in 6 factors. In the 4 factors model, SMB is the least important and the difference between SMB and UMD is astronomic   

In [329]:
tangency_weights(factor_exc_ret).sort_values(by='Tangent Weights', ascending = False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,Tangent Weights
CMA,0.3693
RMW,0.3074
MKT,0.2039
UMD,0.0938
SMB,0.0874
HML,-0.0618


In [330]:
factor_exc_ret_no_rmw_no_cma = factor_exc_ret[['MKT', 'SMB', 'HML','UMD']]
tangency_weights(factor_exc_ret_no_rmw_no_cma).sort_values(by = 'Tangent Weights', ascending = False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,Tangent Weights
MKT,0.3564
HML,0.3489
UMD,0.3009
SMB,-0.0062


## 3.1 - Test for AQR

## 3.1 a)

In [ ]:
portfolios = port_exc_ret.columns

factor = factor_exc_ret[['MKT', 'HML', 'RMW', 'UMD']]

df_lst= []
for port in portfolios:
    fund_ret = port_exc_ret[port]
    reg = regression_based_performance(factor, fund_ret, constant = True)
    mean = reg[7]
    beta = reg[0][0]
    treynor_ratio = reg[1]
    information_ratio = reg[2]
    alpha = reg[3]
    r_squared = reg[4]
    df_lst.append(pd.DataFrame([[mean,alpha,r_squared,beta,treynor_ratio,information_ratio]],columns=['Mean_Ret','Alpha','R-Squared', 'Beta','Treynor Ratio','Information Ratio'],index = [port]))

In [331]:
portfolios = port_exc_ret.columns

factor = factor_exc_ret[['MKT', 'HML', 'RMW', 'UMD']]

df_lst= []
for port in portfolios:
    fund_ret = port_exc_ret[port]
    reg = regression_based_performance(factor, fund_ret, constant = True)
    df_lst.append(reg)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [332]:
# Note: Avoid running it multiple times because of the concat of the DataFrame
reg_perf_aqr = pd.concat(df_lst)
reg_perf_aqr.head()

,Mean Return,R-squared,Alpha,Tracking Error,Information Ratio,Treynor Ratio,MKTBeta,HMLBeta,RMWBeta,UMDBeta
Agric,0.0897,0.3413,0.0095,0.1765,0.0538,0.1071,0.8378,0.1787,-0.0064,0.0841
Food,0.0997,0.4711,0.0114,0.1098,0.1041,0.1464,0.6809,0.1698,0.5074,0.0451
Soda,0.1088,0.3072,0.0196,0.1845,0.1060,0.1387,0.7847,0.2061,0.4947,-0.0871
Beer,0.1194,0.4267,0.0242,0.1291,0.1873,0.1651,0.7233,0.0252,0.6006,0.0903
Smoke,0.1329,0.2726,0.0353,0.1914,0.1845,0.1803,0.7372,0.2493,0.6574,-0.0268


## 3.1 b)
    We would expect to see (ideally) alpha = 0 if AQR's factor model worked well. The alpha is not very low and it's hard to say that AQR's factor model is doing a good job

In [333]:
MAE = abs(reg_perf_aqr['Alpha']).mean()
print(f'Mean Absolute Error of estimated alphas: {MAE:.7f}')

Mean Absolute Error of estimated alphas: 0.0229954


## 3.2 - Test for all Factor models (include AQR, as well)
    CAPM looks like to fit the best with the lowest alpha (0.0206) and French-Fama 5-factors seems to fit the worst (apha = 0.03127). Even though CAPM has 
    Note: Store the factor model's name and data in a dictionary. Iterate over the dictionary

In [334]:
portfolios = port_exc_ret.columns

factors_dict = {'capm': factor_exc_ret[['MKT']], 'fama-french-3f': factor_exc_ret[['MKT', 'SMB', 'HML']],
               'fama-french-5f': factor_exc_ret[['MKT', 'SMB', 'HML', 'RMW', 'CMA']], 'aqr': factor_exc_ret[['MKT', 'HML', 'RMW', 'UMD']],
               'fama-french-3f-UMD': factor_exc_ret[['MKT', 'SMB', 'HML', 'UMD']],
               'fama-french-5f-UMD': factor_exc_ret[['MKT', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']]}

perf_dict_lst = {}
for fm in factors_dict.keys():
    df_lst= []
    for port in portfolios:
        fund_ret = port_exc_ret[port]
        reg = regression_based_performance(factors_dict[fm], fund_ret, constant = True)
        df_lst.append(reg)
    reg_performance = pd.concat(df_lst)
    perf_dict_lst[fm] = reg_performance

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [335]:
pd.DataFrame()
for fm in factors_dict.keys():
    MAE = abs(perf_dict_lst[fm]['Alpha']).mean()
    print(f'{fm.upper()} MAE: {MAE:.7f}')

CAPM MAE: 0.0206076
FAMA-FRENCH-3F MAE: 0.0242000
FAMA-FRENCH-5F MAE: 0.0312722
AQR MAE: 0.0229954
FAMA-FRENCH-3F-UMD MAE: 0.0225679
FAMA-FRENCH-5F-UMD MAE: 0.0287368


In [361]:
mae_time_series_lst = []
mae_index = []

for fm in factors_dict.keys():
    if fm in ['fama-french-3f', 'fama-french-5f', 'aqr']:
        MAE = abs(perf_dict_lst[fm]['Alpha']).mean()
        mae_index.append(str(fm))
        mae_time_series_lst.append(MAE)
        print(f'{fm.upper()} MAE: {MAE:.7f}')
        
mae_time_series = pd.DataFrame(mae_time_series_lst, index = mae_index, columns = ['TIME SERIES MAE'])
    

FAMA-FRENCH-3F MAE: 0.0242000
FAMA-FRENCH-5F MAE: 0.0312722
AQR MAE: 0.0229954


## 3.3
    Don't compare the Betas of factors across models because it does not make sense.
    Check what has changed from one model to another and compare the MAEs with addition/omission of factors

    Compared to 3F model of FM, 5F model's MAE is higher. Adding RMW and CMA factors to the model did not contribute to explanatory power and the pricing. However, adding UMD to 3F FM and 5F FM decreased the MAE

## 3.4
    High r^2 means the model is suitable for Linear Decomposition. The highest achieved in the models below is ~0.60 which is not high but it's still the best among the other models

In [350]:
for fm in factors_dict.keys():
    rsquared = perf_dict_lst[fm]['R-squared'].mean()
    print(f'{fm.upper()} R-squared: {rsquared:.5f}')

CAPM R-squared: 0.52809
FAMA-FRENCH-3F R-squared: 0.57252
FAMA-FRENCH-5F R-squared: 0.59752
AQR R-squared: 0.57708
FAMA-FRENCH-3F-UMD R-squared: 0.57831
FAMA-FRENCH-5F-UMD R-squared: 0.60345


## 3.5 - Cross Sectional Regression - R^2 is irrelevant. Alpha = 0 matters

In [341]:
time_series_premia = (factor_exc_ret.mean()*12).to_frame('Time Series Premia')
time_series_premia.index = [x+"Beta" for x in time_series_premia.index]

In [343]:
AQRRegression = perf_dict_lst['aqr']
y = AQRRegression['Mean Return']
x = AQRRegression[['MKTBeta','HMLBeta','RMWBeta','UMDBeta']]
AQRRegressionCS = sm.OLS(y,x,missing='drop').fit()
AQR_CS_MAE = abs(AQRRegressionCS.resid).mean()
AQR_CS_premia = AQRRegressionCS.params.to_frame("AQR CS Premia")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [345]:
FF3Regression = perf_dict_lst['fama-french-3f']
y = FF3Regression['Mean Return']
x = FF3Regression[['MKTBeta','SMBBeta','HMLBeta']]
FF3RegressionCS = sm.OLS(y,x,missing='drop').fit()
FF3_CS_MAE = abs(FF3RegressionCS.resid).mean()
FF3_CS_Premia = FF3RegressionCS.params.to_frame("FF3 CS Premia")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [346]:
FF5Regression = perf_dict_lst['fama-french-5f']
y = FF5Regression['Mean Return']
x = FF5Regression[['MKTBeta','SMBBeta','HMLBeta','RMWBeta','CMABeta']]
FF5RegressionCS = sm.OLS(y,x,missing='drop').fit()
FF5_CS_MAE = abs(FF5RegressionCS.resid).mean()
FF5_CS_Premia = FF5RegressionCS.params.to_frame("FF5 CS Premia")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [347]:
pd.concat([time_series_premia,AQR_CS_premia,FF3_CS_Premia,FF5_CS_Premia],axis = 1).fillna('')

,Time Series Premia,AQR CS Premia,FF3 CS Premia,FF5 CS Premia
MKTBeta,0.0846,0.0876,0.1016,0.0957
SMBBeta,0.0112,,-0.0646,-0.0577
HMLBeta,0.0253,-0.0398,-0.0175,-0.0335
RMWBeta,0.0465,0.0444,,0.0359
CMABeta,0.0325,,,-0.0152
UMDBeta,0.0609,0.0534,,


In [369]:
MAE_CS = pd.DataFrame([AQR_CS_MAE,FF3_CS_MAE, FF5_CS_MAE], index = mae_time_series.index)
MAE = pd.concat([MAE_CS, mae_time_series], axis = 1)
MAE.columns = ['Cross Section MAE', 'Time Series MAE']
MAE

,Cross Section MAE,Time Series MAE
fama-french-3f,0.0164,0.0242
fama-french-5f,0.0150,0.0313
aqr,0.0130,0.0230
